# インデクサープルAPIを使用したAzure AI Searchでのドキュメントレベルアクセス制御

Azure AI Searchでは、インデクサーを使用してコンテンツを検索インデックスにプルしてインデックス化できます。このノートブックでは、Azure Storage Data Lake Storage (ADLS) Gen2でアクセス制御リスト (ACL) を持つBlobをインデックス化し、ユーザーが表示を許可されている結果のみを返すようにインデックスをクエリする方法を示します。

クエリアクセストークンの背後にあるセキュリティプリンシパルが「ユーザー」を決定します。フォルダーとファイルのACLは、ユーザーがコンテンツの表示を許可されているかどうかを決定し、そのメタデータは他のドキュメントコンテンツと共にインデックスにプルされます。クエリ時に、検索エンジンは内部的にオブジェクトIDに関連付けられていないドキュメントをフィルタリングします。

この機能は現在プレビュー段階です。

あらゆるデータをインデックス化するためのプッシュAPIを使用した代替アプローチについては、[Quickstart-Document-Permissions-Push-API](../Quickstart-Document-Permissions-Push-API/document-permissions-push-api.ipynb)を参照してください。


## 前提条件

+ Azure AI Search、基本階層以上、[システム割り当てマネージドID](https://learn.microsoft.com/azure/search/search-howto-managed-identities-data-sources)と[ロールベースのアクセス制御](https://learn.microsoft.com/azure/search/search-security-enable-roles)を持つもの。

+ Azure Storage、汎用アカウント、[階層名前空間](https://learn.microsoft.com/azure/storage/blobs/create-data-lake-storage-account)を持つもの。

+ フォルダーとファイル、各ファイルに[アクセス制御リストが指定](https://learn.microsoft.com/azure/storage/blobs/data-lake-storage-access-control)されているもの。グループIDの使用を推奨します。

## 権限

このチュートリアルではMicrosoft Entra ID認証と認可を使用します。

+ Azure Storageでは、ローカルでテストしているため、検索サービスIDとユーザーアカウントの両方に**Storage Blob Data Reader**権限が必要です。サンプルにはコンテナーとその内容を作成・設定するコードが含まれているため、**Storage Blob Data Contributor**も必要です。

+ Azure AI Searchでは、オブジェクトを作成してクエリを実行するために**Search Service Contributor**、**Search Index Data Contributor**、**Search Index Data Reader**権限を自分に割り当ててください。詳細については、[ロールを使用してAzure AI Searchに接続する](https://learn.microsoft.com/azure/search/search-security-rbac)と[クイックスタート: ローカルテスト用にキーなしで接続する](https://learn.microsoft.com/azure/search/search-get-started-rbac)を参照してください。

## 制限事項

+ 解析インデクサーオプションは現在サポートされていません。CSV、JSON、またはMarkdown解析のサポートはありません。

## 接続の設定

`sample.env`ファイルを`.env`として保存し、環境変数をAzureエンドポイントを使用するように変更してください。すべての変数を指定する必要があります。

以下のエンドポイントが必要です:

+ Azure AI Search
+ Azure Storage

Azure AI Searchの場合、[Azureポータル](https://portal.azure.com)の概要ページの**基本情報**セクションでエンドポイントを見つけてください。

Azure Storageの場合、[ストレージアカウント設定情報の取得](https://learn.microsoft.com/azure/storage/common/storage-account-get-info)のガイダンスに従って、`.env`ファイル内のすべての変数を指定してください。


## 接続の読み込み

このサンプルコードを実行するには、仮想環境を作成することをお勧めします。Visual Studio Codeで、コントロールパレット（Ctrl+Shift+P）を開いて環境を作成してください。このノートブックはPython 3.10でテストされています。

環境が作成されたら、環境変数を読み込んで接続とオブジェクト名を設定します。

In [ ]:
!az login

In [ ]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import os

load_dotenv(override=True) # take environment variables from .env.

# The following variables from your .env file are used in this notebook
endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
credential = DefaultAzureCredential()
index_name = os.getenv("AZURE_SEARCH_INDEX", "document-permissions-indexer-idx")
indexer_name = os.getenv("AZURE_SEARCH_INDEXER", "document-permissions-indexer-idxr")
datasource_name = os.getenv("AZURE_SEARCH_DATASOURCE", "document-permissions-indexer-ds")
adls_gen2_account_name = os.getenv("AZURE_STORAGE_ACCOUNT_NAME")
adls_gen2_container_name = os.getenv("AZURE_STORAGE_CONTAINER_NAME")
adls_gen2_connection_string = os.environ["AZURE_STORAGE_CONNECTION_STRING"]
adls_gen2_resource_id = os.environ["AZURE_STORAGE_RESOURCE_ID"]
token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")

## インデックスの作成

検索インデックスには、コンテンツと権限メタデータのフィールドを含める必要があります。新しい権限フィルターオプションを文字列フィールドに割り当て、フィールドがフィルタリング可能であることを確認してください。検索エンジンはクエリ時に内部的にフィルターを構築します。

In [ ]:
from azure.search.documents.indexes.models import SearchField, SearchIndex, PermissionFilter, SearchIndexPermissionFilterOption
from azure.search.documents.indexes import SearchIndexClient

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index = SearchIndex(
    name=index_name,
    fields=[
        SearchField(name="id", type="Edm.String", key=True, filterable=True, sortable=True),
        SearchField(name="content", type="Edm.String", searchable=True, filterable=False, sortable=False),
        SearchField(name="oids", type="Collection(Edm.String)", filterable=True, permission_filter=PermissionFilter.USER_IDS),
        SearchField(name="groups", type="Collection(Edm.String)", filterable=True, permission_filter=PermissionFilter.GROUP_IDS),
        SearchField(name="metadata_storage_path", type="Edm.String", searchable=True),
        SearchField(name="metadata_storage_name", type="Edm.String", searchable=True)
    ],
    permission_filter_option=SearchIndexPermissionFilterOption.ENABLED
)

index_client.create_or_update_index(index=index)
print(f"Index '{index_name}' created with permission filter option enabled.")

## データソースの作成

インデクサーが権限メタデータを取得することを認識できるように、`IndexerPermissionOption`を設定します。

In [ ]:
from azure.search.documents.indexes.models import SearchIndexerDataSourceConnection, SearchIndexerDataSourceType, IndexerPermissionOption, SearchIndexerDataContainer, DataSourceCredentials
from azure.search.documents.indexes import SearchIndexerClient
indexer_client = SearchIndexerClient(endpoint=endpoint, credential=credential)
datasource = SearchIndexerDataSourceConnection(
    name=datasource_name,
    type=SearchIndexerDataSourceType.ADLS_GEN2,
    connection_string=f"ResourceId={adls_gen2_resource_id};",
    container=SearchIndexerDataContainer(name=adls_gen2_container_name),
    indexer_permission_options=[IndexerPermissionOption.GROUP_IDS]
)

indexer_client.create_or_update_data_source_connection(datasource)
print(f"Datasource '{datasource_name}' created with permission filter option enabled.")

## グループIDの取得

このステップでは、Graph APIを呼び出してMicrosoft Entra IDのグループIDをいくつか取得します。あなたのグループIDは、次のステップで作成されるオブジェクトのアクセス制御リストに追加されます。2つのグループ識別子が取得され、それぞれ異なるファイルに割り当てられます。

In [ ]:
from msgraph import GraphServiceClient
client = GraphServiceClient(credentials=credential, scopes=["https://graph.microsoft.com/.default"])

groups = await client.me.member_of.get()
first_group_id = groups.value[0].id
second_group_id = groups.value[1].id

## サンプルディレクトリとファイルのアップロード

このステップでは、コンテナー、フォルダーを作成し、ファイルをAzure Storageにアップロードします。各ファイルのアクセス制御リストにあなたのグループIDを割り当てます。

In [ ]:
from azure.storage.filedatalake import DataLakeServiceClient
import requests

service = DataLakeServiceClient.from_connection_string(adls_gen2_connection_string, credential=credential)
container = service.get_file_system_client(adls_gen2_container_name)
if not container.exists():
    container.create_file_system()
root_dir_client = container.get_directory_client("/")
state_parks_dir_client = container.get_directory_client("state-parks")
state_parks_dir_client.create_directory()
root_dir_client.update_access_control_recursive(f"group:{first_group_id}:rwx")
root_dir_client.update_access_control_recursive(f"group:{second_group_id}:rwx")

oregon_dir_client = state_parks_dir_client.create_sub_directory("oregon")
oregon_dir_client.create_directory()
file_client = oregon_dir_client.create_file("oregon_state_parks.csv")
oregon_state_parks_content = requests.get("https://raw.githubusercontent.com/Azure-Samples/azure-search-sample-data/refs/heads/main/state-parks/Oregon/oregon_state_parks.csv").content.decode("utf-8")
file_client.upload_data(oregon_state_parks_content, overwrite=True)
oregon_dir_client.update_access_control_recursive(f"group:{first_group_id}:rwx")

washington_dir_client = state_parks_dir_client.create_sub_directory("washington")
washington_dir_client.create_directory()
file_client = washington_dir_client.create_file("washington_state_parks.csv")
washington_state_parks_content = requests.get("https://raw.githubusercontent.com/Azure-Samples/azure-search-sample-data/refs/heads/main/state-parks/Washington/washington_state_parks.csv").content.decode("utf-8")
file_client.upload_data(washington_state_parks_content, overwrite=True)
washington_dir_client.update_access_control_recursive(f"group:{second_group_id}:rwx")

## インデクサーの実行

インデクサーを開始して、データ取得からインデックス化まですべての操作を実行します。接続エラーや権限の問題はここで明らかになります。

In [ ]:
from azure.search.documents.indexes.models import SearchIndexer, FieldMapping

indexer = SearchIndexer(
    name=indexer_name,
    target_index_name=index_name,
    data_source_name=datasource_name,
    field_mappings=[
        FieldMapping(source_field_name="metadata_group_ids", target_field_name="groups"),
        FieldMapping(source_field_name="metadata_user_ids", target_field_name="oids"),
    ]
)

indexer_client.create_or_update_indexer(indexer)
print(f"Indexer '{indexer_name}' created")


## x-ms-query-source-authorizationを使用したサンプルデータの検索

クエリを実行する前に、インデクサーの処理が完了するまで待ってください。このクエリでは、無条件検索のために空の検索文字列（`*`）を使用します。ファイル名と各ファイルに関連付けられた権限メタデータを返します。各ファイルが異なるグループIDに関連付けられていることに注意してください。

In [ ]:
from azure.search.documents import SearchClient
search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential)

results = search_client.search(search_text="*", x_ms_query_source_authorization=token_provider(), select="metadata_storage_path,oids,groups", order_by="id asc")
for result in results:
    print(f"Path: {result['metadata_storage_path']}, OID: {result['oids']}, Group: {result['groups']}")

In [ ]:
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")

In [ ]:
from azure.search.documents import SearchClient
search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential)

results = search_client.search(search_text="*", x_ms_query_source_authorization=token_provider(), select="metadata_storage_path,oids,groups", order_by="id asc")
for result in results:
    print(f"Path: {result['metadata_storage_path']}, OID: {result['oids']}, Group: {result['groups']}")

## x-ms-query-source-authorizationなしでのサンプルデータの検索

このステップでは、認証が失敗した場合のユーザーエクスペリエンスを示します。レスポンスには結果が返されません。

In [ ]:
from azure.search.documents import SearchClient
search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential)

results = search_client.search(search_text="*", x_ms_query_source_authorization=None, select="metadata_storage_path,oids,groups", order_by="id asc")
for result in results:
    print(f"Path: {result['metadata_storage_path']}, OID: {result['oids']}, Group: {result['groups']}")

In [ ]:
from azure.search.documents import SearchClient
search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential)

results = search_client.search(search_text="*", x_ms_query_source_authorization=None, select="metadata_storage_path,oids,groups", order_by="id asc")
for result in results:
    print(f"Path: {result['metadata_storage_path']}, OID: {result['oids']}, Group: {result['groups']}")

## 次のステップ

詳細については、[Azure AI Searchでのドキュメントレベルアクセス制御](https://learn.microsoft.com/azure/search/search-document-level-access-overview)を参照してください。